In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
model = Sequential()

Now just remember the architecture in mind and start adding the layers into the network. our Kernel_size is (3,3) and the pool_size is (2,2) always.

In [ ]:
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(1, activation='sigmoid', name='output'))


In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
vgg.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

To use a pre-trained model just makes the middle layers non trainable and remove the last layer.

We are also going to do the same let’s makes the middle layers freeze.

In [ ]:
for layer in vgg.layers:
  layer.trainable = False

We have already removed the output layer by include_top = False.

Let’s add our own output layer with only one node.

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(1, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Now we need to compile our model so that we can train it. our problem is a binary classification problem so we are going to use binary_crossentropy and optimizer we are going to use as Adam and for metrics, we are going to use accuracy.

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  #optimizer='adam',
  metrics=['accuracy']
)

Let’s generate training and validation data using the data generator

In [ ]:
train_path="/content/gdrive/My Drive/datasets/Buildings224/train"
test_path="/content/gdrive/My Drive/datasets/Buildings224/test"

train_datagen = ImageDataGenerator( validation_split= 0.1,
                                    rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

validation_set = train_datagen.flow_from_directory(train_path, 
                                                subset='validation')

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')



Found 240 images belonging to 5 classes.
Found 2415 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


Train the model

In [ ]:
r = model.fit(
  training_set,  
  #validation_data=validation_set,
  epochs=12,
  steps_per_epoch=len(training_set)
)

Epoch 1/12
151/151 [==============================] - 1245s 8s/step - loss: 0.0869 - accuracy: 0.7755
Epoch 2/12
151/151 [==============================] - 1252s 8s/step - loss: 0.0799 - accuracy: 0.7941
Epoch 3/12
151/151 [==============================] - 1250s 8s/step - loss: 0.0604 - accuracy: 0.7347
Epoch 4/12
151/151 [==============================] - 1248s 8s/step - loss: 0.0685 - accuracy: 0.7636
Epoch 5/12
151/151 [==============================] - 1246s 8s/step - loss: 0.0877 - accuracy: 0.8131
Epoch 6/12
151/151 [==============================] - 1248s 8s/step - loss: 0.0501 - accuracy: 0.8505
Epoch 7/12
151/151 [==============================] - 1254s 8s/step - loss: 0.0726 - accuracy: 0.9000
Epoch 8/12
151/151 [==============================] - 1257s 8s/step - loss: 0.0388 - accuracy: 0.9154
Epoch 9/12
151/151 [==============================] - 1254s 8s/step - loss: 0.0430 - accuracy: 0.8850
Epoch 10/12
151/151 [==============================] - 1249s 8s/step - loss: 0.053

Test the model

In [ ]:
model.save('/content/gdrive/My Drive/datasets/Buildings224/model91')


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/datasets/Buildings224/model91/assets


In [ ]:
model.save('/content/gdrive/My Drive/datasets/Buildings224/model91lite')


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/datasets/Buildings224/model91lite/assets


In [ ]:
tf.float16


NameError: ignored

In [ ]:
import pathlib
tflite_models_dir = pathlib.Path("/content/gdrive/My Drive/datasets/Buildings224/model91lite/")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_fp16_model = converter.convert()
tflite_model_fp16_file = tflite_models_dir/"model_quant_f16.tflite"
tflite_model_fp16_file.write_bytes(tflite_fp16_model)

INFO:tensorflow:Assets written to: /tmp/tmpk704_zii/assets


I0518 22:59:49.574575 139936046589824 builder_impl.py:775] Assets written to: /tmp/tmpk704_zii/assets


29692880

In [ ]:
interpreter_fp16 = tf.lite.Interpreter(model_path=str(tflite_model_fp16_file))
interpreter_fp16.allocate_tensors()

In [ ]:
test_image = np.expand_dims(test_set[0], axis=0).astype(np.float32)

input_index = interpreter_fp16.get_input_details()[0]["index"]
output_index = interpreter_fp16.get_output_details()[0]["index"]

interpreter_fp16.set_tensor(input_index, test_image)
interpreter_fp16.invoke()
predictions = interpreter_fp16.get_tensor(output_index)

plt.imshow(test_set[0])
template = "True:{true}, predicted:{predict}"
_ = plt.title(template.format(true= str(test_labels[0]),
                              predict=str(np.argmax(predictions[0]))))
plt.grid(False)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


ValueError: ignored

Ricarico il modello per il test:


In [ ]:
new_model = tf.keras.models.load_model('/content/gdrive/My Drive/datasets/Buildings224/model91')


ValueError: ignored

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

#model.evaluate_generator(test_set)
Y_pred = model.predict_generator(test_set, len(test_set)) #check generator aka predict()
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['Arch_Bridges', 'Castles', 'Churches', 'Towers', 'Triumphal_Arches']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix
[[12 11  9  6 12]
 [15  9  8 10  8]
 [ 6 15 11 13  5]
 [10  8 12 11  9]
 [ 6 12 11  8 13]]
Classification Report
                  precision    recall  f1-score   support

    Arch_Bridges       0.24      0.24      0.24        50
         Castles       0.16      0.18      0.17        50
        Churches       0.22      0.22      0.22        50
          Towers       0.23      0.22      0.22        50
Triumphal_Arches       0.28      0.26      0.27        50

        accuracy                           0.22       250
       macro avg       0.23      0.22      0.22       250
    weighted avg       0.23      0.22      0.22       250



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

#Confution Matrix and Classification Report
Y_pred = model.predict_generator(training_set, len(training_set)) #check generator alternative aka predict()
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(training_set.classes, y_pred))
print('Classification Report')
target_names = ['Arch_Bridges', 'Castles', 'Churches', 'Towers', 'Triumphal_Arches']
print(classification_report(training_set.classes, y_pred, target_names=target_names))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


KeyboardInterrupt: ignored